# 🧪 Speed Test - 10K Samples, 0 Workers

Test tốc độ thuần GPU với Mixed Precision

---

In [ ]:
# 1. Clone repo
!rm -rf EV_Translate_Modle_NLP_Project
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# 2. Install deps
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml

In [ ]:
# 3. Create .env
HF_TOKEN = "YOUR_HF_TOKEN"
with open('.env', 'w') as f:
    f.write(f'HF_TOKEN={HF_TOKEN}\n')
print('✓ .env created')

In [ ]:
# 4. Download data
!python scripts/download_phomt.py

In [ ]:
# 5. Preprocess 10K samples
!python scripts/preprocess_data.py --max-samples 10000

In [ ]:
# 6. Config for speed test: num_workers=0, processed data, 2 epochs
import yaml

with open('config/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Speed test settings
cfg['data']['source'] = 'processed'
cfg['training']['epochs'] = 2
cfg['hardware']['num_workers'] = 0  # No workers = no overhead

with open('config/config.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config: 10K samples, 0 workers, 2 epochs')

In [ ]:
# 7. Verify GPU
import torch
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 8. Train! Watch the it/s speed
from src.train import Trainer

trainer = Trainer()
trainer.setup()
trainer.train()

print("\n✅ Training complete!")
print(f"Mixed Precision: {trainer.use_amp}")

In [ ]:
# 9. Test translation
from src.evaluate import load_translator

translator = load_translator()

for s in ["Hello", "How are you?", "Thank you"]:
    print(f"EN: {s} -> VI: {translator.translate(s)}")

## 📊 Expected Results

With GPU + AMP + 0 workers:
- **Speed: 50-100+ it/s**
- **1 epoch: ~5-10 seconds**

If much slower, check GPU is enabled.